### Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import pickle

import pandas as pd

from ase import io

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/seoin_irox_data/process_bulk_data


In [2]:
dir_root = os.path.join(
    os.environ["PROJ_DATA"],
    "PROJ_IrOx_OER/seoin_irox_data/bulk",
    )

In [3]:
bulk_systems_dict = {
    
    "iro2": [
        "anatase-fm",
        "brookite-fm",
        "columbite-fm",
        "pyrite-fm",
        "rutile-fm",
        ],

    "iro3": [
        "Amm2",
        "cmcm",
        "pm-3m",
        ],

    }

crystal_rename_dict = {
    'anatase-fm': 'anatase',
    'brookite-fm': 'brookite',
    'columbite-fm': 'columbite',
    'pyrite-fm': 'pyrite',
    'rutile-fm': 'rutile',
    'Amm2': 'amm2',
    'cmcm': 'cmcm',
    'pm-3m': 'pm-3m',
    }

In [4]:
def calc_dH(
    e_per_atom,
    stoich=None,
    num_H_atoms=0,
    ):
    """
    The original method is located in:
    F:\Dropbox\01_norskov\00_git_repos\PROJ_IrOx_Active_Learning_OER\data\proj_data_irox.py

    Based on a E_DFT/atom of -7.047516 for rutile-IrO2

    See the following dir for derivation:
        PROJ_IrOx_Active_Learning_OER/workflow/energy_treatment_deriv/calc_references
    """
    # | - calc_dH
    o_ref = -4.64915959
    ir_metal_fit = -9.32910211636731
    h_ref = -3.20624595

    if stoich == "AB2":
        dH = (2 + 1) * e_per_atom - 2 * o_ref - ir_metal_fit
        dH_per_atom = dH / 3.
    elif stoich == "AB3":
        dH = (3 + 1) * e_per_atom - 3 * o_ref - ir_metal_fit
        dH_per_atom = dH / 4.

    elif stoich == "IrHO3" or stoich == "IrO3H" or stoich == "iro3h" or stoich == "iroh3":
        dH = (3 + 1 + 1) * e_per_atom - 3 * o_ref - ir_metal_fit - h_ref
        dH_per_atom = dH / 5.


    return(dH_per_atom)
    #__|

In [5]:
data_dict_list = []
for bulk_i, polymorphs_i in bulk_systems_dict.items():
    # print(bulk_i)
    
    if bulk_i == "iro2":
        stoich_i = "AB2"
    elif bulk_i == "iro3":
        stoich_i = "AB3"


    for poly_j in polymorphs_i:
        # print("  ", poly_j, sep="")

        # poly_new_j = crystal_rename_dict.get(poly_j, poly_j)
        poly_new_j = crystal_rename_dict.get(poly_j, "TEMP")


        path_rel_j = os.path.join(
            bulk_i,
            poly_j)
        path_j = os.path.join(
            dir_root,
            path_rel_j)


        # #################################################
        # Reading OUTCAR atoms
        atoms_j = io.read(os.path.join(path_j, "OUTCAR"))

        pot_e_j = atoms_j.get_potential_energy()

        volume_j = atoms_j.get_volume()
        num_atoms_j = atoms_j.get_global_number_of_atoms()

        volume_pa_j = volume_j / num_atoms_j

        # print(atoms_j.get_chemical_formula())

        # #################################################
        # Calc formation energy
        dH_i = calc_dH(
            pot_e_j / num_atoms_j,
            stoich=stoich_i,
            num_H_atoms=0,
            )

        # #################################################
        data_dict_j = dict()
        # #################################################
        data_dict_j["stoich"] = stoich_i
        # data_dict_j["crystal"] = poly_j
        data_dict_j["crystal"] = poly_new_j
        data_dict_j["dH"] = dH_i
        data_dict_j["volume"] = volume_j
        data_dict_j["num_atoms"] = num_atoms_j
        data_dict_j["volume_pa"] = volume_pa_j
        data_dict_j["atoms"] = atoms_j
        data_dict_j["path"] = path_rel_j
        # #################################################
        data_dict_list.append(data_dict_j)
        # #################################################

# #########################################################
df = pd.DataFrame(
    data_dict_list
    )
# #########################################################

In [6]:
df_seoin_bulk = df

# Pickling data ###########################################
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/seoin_irox_data/process_bulk_data",
    "out_data")
if not os.path.exists(directory):
    os.makedirs(directory)

with open(os.path.join(directory, "df_seoin_bulk.pickle"), "wb") as fle:
    pickle.dump(df_seoin_bulk, fle)
# #########################################################

In [7]:
df

,stoich,crystal,dH,volume,num_atoms,volume_pa,atoms,path
0,AB2,anatase,-0.570008,150.801160,12,12.566763,"(Atom('Ir', [-0.0, 0.0, 4.88876], index=0), At...",iro2/anatase-fm
1,AB2,brookite,-0.663728,280.231681,24,11.676320,"(Atom('Ir', [0.70294, 1.20467, 4.93099], index...",iro2/brookite-fm
2,AB2,columbite,-0.746275,129.995983,12,10.832999,"(Atom('Ir', [0.00014, 0.80226, 3.96457], index...",iro2/columbite-fm
3,AB2,pyrite,-0.739150,119.972033,12,9.997669,"(Atom('Ir', [-0.0, 0.0, 0.0], index=0), Atom('...",iro2/pyrite-fm
4,AB2,rutile,-0.834534,65.725895,6,10.954316,"(Atom('Ir', [2.27005, 2.27005, 1.59419], index...",iro2/rutile-fm
5,AB3,amm2,-0.532592,203.893673,16,12.743355,"(Atom('Ir', [1.871, 1.84295, 5.53954], index=0...",iro3/Amm2
6,AB3,cmcm,-0.573335,382.619000,32,11.956844,"(Atom('Ir', [1.78447, 1.82286, 6e-05], index=0...",iro3/cmcm
7,AB3,pm-3m,-0.445713,55.071412,4,13.767853,"(Atom('Ir', [1.9023, 1.90226, 1.90234], index=...",iro3/pm-3m


In [8]:
#     'amm2',
#     'anatase',
#     'brookite',
#     'cmcm',
#     'columbite',
#     'pm-3m',
#     'pyrite',
#     'rutile',

In [9]:
#     'anatase-fm',
#     'brookite-fm',
#     'columbite-fm',
#     'pyrite-fm',
#     'rutile-fm',
#     'Amm2',
#     'cmcm',
#     'pm-3m',

In [10]:
# df.crystal.tolist()